# 线性回归——使用Gluon

[前一章](linear-regression-scratch.md)我们仅仅使用了`ndarray`和`autograd`来实现线性回归，这一章我们仍然实现同样的模型，但是使用高层抽象包`gluon`。

## 创建数据集

我们生成同样的数据集

In [1]:
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import gluon

data_root = '../data'
txt_root = data_root + '/CephalometricLandmark/AnnotationsByMD'
landmark_index = 15
num_examples = 150
num_inputs = 2
X = nd.zeros(shape=(num_examples, 3, 2))
y = nd.zeros(shape=(num_examples, 2))
for i in range(num_examples):
    txt_filename1 = txt_root + '/400_senior' + "/%03d.txt" % (i+1)
    with open(txt_filename1, 'r') as f:
        txts = f.read().split()
    x7 = int(txts[6].split(',')[0])
    y7 = int(txts[6].split(',')[1])
    X[i][0][0] = x7
    X[i][0][1] = y7
    
    x8 = int(txts[7].split(',')[0])
    y8 = int(txts[7].split(',')[1])
    X[i][1][0] = x8
    X[i][1][1] = y8
    
    x9 = int(txts[8].split(',')[0])
    y9 = int(txts[8].split(',')[1])
    X[i][2][0] = x9
    X[i][2][1] = y9
    
    x16 = int(txts[15].split(',')[0])
    y16 = int(txts[15].split(',')[1])
    y[i][0] = x16
    y[i][1] = y16

print(X)


[[[ 1333.  2200.]
  [ 1263.  2272.]
  [ 1305.  2252.]]

 [[ 1381.  2005.]
  [ 1320.  2091.]
  [ 1363.  2063.]]

 [[ 1460.  1882.]
  [ 1406.  1935.]
  [ 1448.  1920.]]

 [[ 1468.  2016.]
  [ 1418.  2091.]
  [ 1449.  2061.]]

 [[ 1324.  1964.]
  [ 1278.  2010.]
  [ 1309.  1993.]]

 [[ 1388.  2109.]
  [ 1345.  2155.]
  [ 1368.  2140.]]

 [[ 1345.  2143.]
  [ 1287.  2188.]
  [ 1325.  2178.]]

 [[ 1367.  1825.]
  [ 1318.  1886.]
  [ 1354.  1867.]]

 [[ 1307.  2014.]
  [ 1272.  2064.]
  [ 1294.  2045.]]

 [[ 1366.  1876.]
  [ 1332.  1934.]
  [ 1359.  1912.]]

 [[ 1269.  1974.]
  [ 1198.  2035.]
  [ 1244.  2018.]]

 [[ 1620.  2077.]
  [ 1581.  2136.]
  [ 1604.  2109.]]

 [[ 1278.  1942.]
  [ 1234.  2002.]
  [ 1264.  1980.]]

 [[ 1245.  2018.]
  [ 1198.  2066.]
  [ 1229.  2051.]]

 [[ 1292.  2038.]
  [ 1245.  2104.]
  [ 1276.  2079.]]

 [[ 1435.  1956.]
  [ 1393.  2003.]
  [ 1421.  1989.]]

 [[ 1411.  2067.]
  [ 1358.  2138.]
  [ 1389.  2110.]]

 [[ 1249.  1994.]
  [ 1194.  2028.]
  [ 1226.  

## 数据读取

但这里使用`data`模块来读取数据。

In [2]:
batch_size = 10
dataset = gluon.data.ArrayDataset(X, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

读取跟前面一致：

In [3]:
for data, label in data_iter:
    print(data, label)
    break


[[[ 1245.  2018.]
  [ 1198.  2066.]
  [ 1229.  2051.]]

 [[ 1350.  1916.]
  [ 1292.  1965.]
  [ 1333.  1955.]]

 [[ 1300.  2147.]
  [ 1247.  2221.]
  [ 1284.  2197.]]

 [[ 1400.  1859.]
  [ 1351.  1903.]
  [ 1386.  1886.]]

 [[ 1326.  2018.]
  [ 1259.  2063.]
  [ 1303.  2054.]]

 [[ 1431.  1959.]
  [ 1381.  2036.]
  [ 1417.  2003.]]

 [[ 1438.  2018.]
  [ 1366.  2079.]
  [ 1414.  2060.]]

 [[ 1116.  2069.]
  [ 1046.  2124.]
  [ 1086.  2105.]]

 [[ 1288.  2080.]
  [ 1223.  2142.]
  [ 1267.  2124.]]

 [[ 1454.  2008.]
  [ 1397.  2067.]
  [ 1437.  2047.]]]
<NDArray 10x3x2 @cpu(0)> 
[[ 1312.  2105.]
 [ 1456.  1994.]
 [ 1389.  2249.]
 [ 1474.  1942.]
 [ 1369.  2082.]
 [ 1487.  2018.]
 [ 1497.  2079.]
 [ 1206.  2190.]
 [ 1305.  2155.]
 [ 1526.  2083.]]
<NDArray 10x2 @cpu(0)>


## 定义模型

之前一章中，当我们从0开始训练模型时，需要先声明模型参数，然后再使用它们来构建模型。但`gluon`提供大量预定义的层，我们只需要关注使用哪些层来构建模型。例如线性模型就是使用对应的`Dense`层；之所以称为dense层，是因为输入的所有节点都与后续的节点相连。在这个例子中仅有一个输出，但在大多数后续章节中，我们会用到具有多个输出的网络。

我们之后还会介绍如何构造任意结构的神经网络，但对于初学者来说，构建模型最简单的办法是利用`Sequential`来所有层串起来。输入数据之后，`Sequential`会依次执行每一层，并将前一层的输出，作为输入提供给后面的层。首先我们定义一个空的模型：

In [4]:
net = gluon.nn.Sequential()

然后我们加入一个`Dense`层，它唯一必须定义的参数就是输出节点的个数，在线性模型里面是1.

In [5]:
net.add(gluon.nn.Dense(2))

（注意这里我们并没有定义说这个层的输入节点是多少，这个在之后真正给数据的时候系统会自动赋值。我们之后会详细介绍这个特性是如何工作的。）

## 初始化模型参数

在使用前`net`我们必须要初始化模型权重，这里我们使用默认随机初始化方法（之后我们会介绍更多的初始化方法）。

In [6]:
net.initialize()

## 损失函数

`gluon`提供了平方误差函数：

In [7]:
square_loss = gluon.loss.L2Loss()

## 优化

同样我们无需手动实现随机梯度下降，我们可以创建一个`Trainer`的实例，并且将模型参数传递给它就行。

In [8]:
trainer = gluon.Trainer(
    net.collect_params(), 'sgd', {'learning_rate': 0.1})

## 训练
使用`gluon`使模型训练过程更为简洁。我们不需要挨个定义相关参数、损失函数，也不需使用随机梯度下降。`gluon`的抽象和便利的优势将随着我们着手处理更多复杂模型的愈发显现。不过在完成初始设置后，训练过程本身和前面没有太多区别，唯一的不同在于我们不再是调用`SGD`，而是`trainer.step`来更新模型（此处一并省略之前绘制损失变化的折线图和散点图的过程，有兴趣的同学可以自行尝试）。

In [9]:
epochs = 5
batch_size = 10
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
    print("Epoch %d, average loss: %f" % (e, total_loss/num_examples))

Epoch 0, average loss: nan
Epoch 1, average loss: nan
Epoch 2, average loss: nan
Epoch 3, average loss: nan
Epoch 4, average loss: nan


比较学到的和真实模型。我们先从`net`拿到需要的层，然后访问其权重和位移。

In [11]:
dense = net[0]
dense.weight.data()


[[ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]]
<NDArray 2x6 @cpu(0)>

In [ ]:
true_b, dense.bias.data()

## 结论

可以看到`gluon`可以帮助我们更快更干净地实现模型。


## 练习

- 在训练的时候，为什么我们用了比前面要大10倍的学习率呢？（提示：可以尝试运行 `help(trainer.step)`来寻找答案。）
- 如何拿到`weight`的梯度呢？（提示：尝试 `help(dense.weight)`）

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/742)